In [ ]:
#Imports:
import pandas as pd
import os
import json
import numpy as np

In [ ]:
#Formatting dataset:
#Time sequences are 10 timepoints (Messages) with 7 features per message.
#Organized by car.
dataDirect = "Data/"
allDatasets = {}

for filename in os.listdir(dataDirect):
    if filename.endswith(".json"):
       allDatasets[filename.split(".")[0]] = json.load(open(dataDirect+filename))

doSData = allDatasets["DoS_1416"]
fullDoSDataset = {}
for car in doSData:
    carVal = doSData[car]
    bSMs = []
    count = 0
    lastentry = []
    for BSM in carVal["BSMs"].values():
        BSMVals = []
        BSMVals.append()


        for value in BSM.values():
            BSMVals.append(value)
        count+=1
        bSMs.append([BSMVals, carVal["Attack Type"]])
        lastentry = [BSMVals, carVal["Attack Type"]]
    while count % 100:
        count+=1
        bSMs.append(lastentry)
    fullDoSDataset[car] = bSMs
    


print(fullDoSDataset, file=open("out.txt", "w"))
#Split into Training and Test datasets
#Format data for AI

In [ ]:
#Create Model
#Create Learning function
#Create Testing function
#Create tests